# São Paulo Business Analytics - 3 Million Companies

Analyzing the complete São Paulo business ecosystem.  
Data source: Brazilian Federal Revenue (Receita Federal)


In [ ]:
import pandas as pd
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

# Load full São Paulo dataset from GitHub Releases
# Update with your actual release URL after uploading
GITHUB_RELEASE_URL = "https://github.com/caiopizzol/cnpj-data-pipeline/releases/download/v1.0/sample_20250906.parquet"

try:
    print("Loading 3M+ companies from GitHub...")
    df = pd.read_parquet(GITHUB_RELEASE_URL)
    print(f"✅ Loaded {len(df):,} companies")
except Exception:
    # Fallback to local file
    import glob

    files = glob.glob("exports/sp_full_*.parquet")
    if files:
        df = pd.read_parquet(sorted(files)[-1])
        print(f"📁 Using local file: {len(df):,} companies")
    else:
        # Use sample for demo
        files = glob.glob("exports/sample_*.parquet")
        if files:
            df = pd.read_parquet(sorted(files)[-1])
            print(f"⚠️ Using sample: {len(df):,} companies")
            print("   For full data: python export.py sp_full")
        else:
            raise FileNotFoundError("No data found. Run: python export.py sample")

print(f"Memory: {df.memory_usage(deep=True).sum() / 1024**2:.0f} MB")

Loading 3M+ companies from GitHub...
✅ Loaded 10,000 companies
Memory: 4 MB


## Key Insights


In [2]:
# 1. Business Survival
df["data_inicio_atividade"] = pd.to_datetime(df["data_inicio_atividade"])
df["age_years"] = (datetime.now() - df["data_inicio_atividade"]).dt.days / 365.25

survival_rate = (df["age_years"] > 5).mean() * 100
print(f"Only {survival_rate:.1f}% of companies survive more than 5 years")

Only 100.0% of companies survive more than 5 years


In [3]:
# 2. Industry Concentration
top_industries = df["cnae_fiscal_principal"].value_counts().head(10)
concentration = top_industries.sum() / len(df) * 100
print(f"Top 10 industries = {concentration:.1f}% of all companies")
print("\nTop 5 CNAEs:")
for cnae, count in top_industries.head(5).items():
    pct = count / len(df) * 100
    print(f"  {cnae}: {count:,} companies ({pct:.1f}%)")

Top 10 industries = 29.7% of all companies

Top 5 CNAEs:
  8112500: 615 companies (6.2%)
  9430800: 599 companies (6.0%)
  9491000: 571 companies (5.7%)
  4781400: 250 companies (2.5%)
  5611203: 244 companies (2.4%)


In [4]:
# 3. Capital Distribution
capital_by_size = df.groupby("porte")["capital_social"].agg(["mean", "median", "count"])
print("Capital by company size:")
print(capital_by_size)

Capital by company size:
               mean   median  count
porte                              
01     1.353596e+05      0.0   3888
03     2.604410e+05  30000.0   1109
05     1.661051e+09      0.0   5003


In [5]:
# 4. Geographic Insights
cities = df["municipio"].value_counts()
top_city_pct = cities.iloc[0] / len(df) * 100
print(f"\nSão Paulo capital: {top_city_pct:.1f}% of all companies")
print(f"Companies spread across {len(cities)} cities")


São Paulo capital: 33.6% of all companies
Companies spread across 547 cities


## Your Analysis

```python
# Filter by your industry
my_cnae = '6201500'  # Software
competitors = df[df['cnae_fiscal_principal'] == my_cnae]

# Find companies in your city
my_city = '3550308'  # São Paulo
local = df[df['municipio'] == my_city]
```

---

Data source: [cnpj-data-pipeline](https://github.com/caiopizzol/cnpj-data-pipeline)
